In [6]:
!pip install stable_baselines3
!pip install 'shimmy>=0.2.1'

In [2]:

import gym
from stable_baselines3.common.vec_env import DummyVecEnv
import numpy as np
from stable_baselines3 import DQN, PPO, A2C
import pygame
import threading
import time
import gym
from gym import spaces
import numpy as np
import pygame
import gym
from stable_baselines3.common.vec_env import DummyVecEnv
import numpy as np
from stable_baselines3 import DQN, PPO, A2C
import random
import matplotlib.pyplot as plt

class PongEnvironment(gym.Env):
    def __init__(self):
        super(PongEnvironment, self).__init__()
        self.width = 400
        self.height = 300
        self.ball_radius = 10
        self.paddle_width = 10
        self.paddle_height = 60
        self.paddle_offset = 20
        self.ball_pos = np.array([self.width // 2, self.height // 2], dtype=float)
        self.ball_vel = np.array([5, 1], dtype=float) * 100
        self.paddle_pos = self.height // 2
        self.screen = pygame.display.set_mode((self.width, self.height))
        pygame.display.set_caption("Pong")

        self.action_space = spaces.Discrete(2)
        self.observation_space = spaces.Box(low=0, high=1, shape=(5,), dtype=np.float32)

    def reset(self):
        self.time_step=0
        self.start_time=time.time()
        self.tot_reward=0
        self.ball_pos = np.array([self.width // 2, self.height // random.randrange(1,10)], dtype=float)
        self.ball_vel = np.array([random.random(), random.random()], dtype=float) * 100
        self.paddle_pos = self.height // 2
        return self.get_state()

    def step(self, action):

        self.time_step+=1
        if action == 0:
            move = -5
        else:
            move = 5
        self.move_paddle(move)
        self.ball_pos += self.ball_vel

        if (
            self.ball_pos[1] <= self.ball_radius
            or self.ball_pos[1] >= self.height - self.ball_radius
        ):
            self.ball_vel[1] *= -1

        reward = 0
        done = False

        if self.ball_pos[0] <= self.paddle_width + self.ball_radius:
            if (
                self.paddle_pos - self.paddle_height / 2
                <= self.ball_pos[1]
                <= self.paddle_pos + self.paddle_height / 2
            ):
                self.ball_vel[0] *= -1
                reward = 1
                self.tot_reward+=1
            else:
                reward = 0
                done = True
        if self.tot_reward>10 or time.time()-self.start_time>30:
                print('xxx')
                done = True
        if self.ball_pos[0] >= self.width - self.ball_radius - self.paddle_width:
            self.ball_vel[0] *= -1

        return self.get_state(), reward, done, {}

    def move_paddle(self, move):
        self.paddle_pos = np.clip(
            self.paddle_pos + move,
            self.paddle_height / 2,
            self.height - self.paddle_height / 2,
        )

    def get_state(self):
        return np.array(
            [
                self.ball_pos[0] / self.width,
                self.ball_pos[1] / self.height,
                self.ball_vel[0] / 100,
                self.ball_vel[1] / 100,
                self.paddle_pos / self.height,
            ]
        )

    def render(self, mode="rgb_array"):
        self.screen.fill((0, 0, 0))
        pygame.draw.rect(
            self.screen,
            (255, 255, 255),
            pygame.Rect(
                0,
                int(self.paddle_pos - self.paddle_height / 2),
                self.paddle_width,
                self.paddle_height,
            ),
        )
        pygame.draw.circle(
            self.screen,
            (255, 255, 255),
            (int(self.ball_pos[0]), int(self.ball_pos[1])),
            self.ball_radius,
        )
        pygame.display.update()

        if mode == "rgb_array":
            data = pygame.surfarray.array3d(pygame.display.get_surface())
            return np.transpose(data, (1, 0, 2))
        return np.transpose(data, (1, 0, 2))



ImportError: cannot import name 'get_device' from partially initialized module 'stable_baselines3.common.utils' (most likely due to a circular import) (c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\utils.py)

In [29]:
class DQNAgent:
    def __init__(self, env):
        self.model = DQN("MlpPolicy", env, verbose=1)
        self.env=env

    def train(self, total_timesteps):
        self.model.learn(total_timesteps=total_timesteps)
    def calculate_expertise(self, num_episodes=100):
        total_rewards = 0
        total_rewards_list=[]
        for zz in range(num_episodes):
            obs = self.env.reset()
            done = False
            episode_reward = 0

            while not done and episode_reward<100:
                action, _ = self.model.predict(obs)
                obs, reward, done, _ = self.env.step(action)
                episode_reward += reward
            total_rewards += episode_reward
            print(episode_reward)
            total_rewards_list.append(episode_reward)

        average_reward = total_rewards / num_episodes
        return average_reward,total_rewards_list


class PPOAgent:
    def __init__(self, env):
        self.model = PPO("MlpPolicy", env, verbose=1)
        self.env=env


    def train(self, total_timesteps):
        self.model.learn(total_timesteps=total_timesteps)
    def calculate_expertise(self, num_episodes=100):
        total_rewards = 0
        total_rewards_list=[]
        for zz in range(num_episodes):
            obs = self.env.reset()
            done = False
            episode_reward = 0

            while not done and total_rewards<100:
                action, _ = self.model.predict(obs)
                obs, reward, done, _ = self.env.step(action)
                episode_reward += reward

            total_rewards += episode_reward
            print(episode_reward)
            total_rewards_list.append(episode_reward)
            print(zz)

        average_reward = total_rewards / num_episodes
        return average_reward,total_rewards_list


class A2CAgent:
    def __init__(self, env):
        self.model = A2C("MlpPolicy", env, verbose=1)
        self.env=env

    def train(self, total_timesteps):
        self.model.learn(total_timesteps=total_timesteps)

    def calculate_expertise(self, num_episodes=100):
        total_rewards = 0
        total_rewards_list=[]
        for zz in range(num_episodes):
            obs = self.env.reset()
            done = False
            episode_reward = 0

            while not done and episode_reward<100:
                action, _ = self.model.predict(obs)
                obs, reward, done, _ = self.env.step(action)
                episode_reward += reward

            total_rewards += episode_reward
            total_rewards_list.append(episode_reward)

        average_reward = total_rewards / num_episodes
        return average_reward,total_rewards_list
all_rewards=[[],[],[]]
for _ in range(10):

  env = DummyVecEnv([lambda: PongEnvironment()])

  dqn_agent = DQNAgent(env)
  ppo_agent = PPOAgent(env)
  a2c_agent = A2CAgent(env)




  # dqn_agent.model = DQN.load('./dqn.zip')
  # ppo_agent.model = PPO.load('./ppo.zip')
  # a2c_agent.model = A2C.load('./a2c.zip')


  total_timesteps = 100000
  dqn_agent.train(total_timesteps)
  ppo_agent.train(total_timesteps)
  a2c_agent.train(total_timesteps)


  dqn_a_r,dqn_r_l = dqn_agent.calculate_expertise(100)
  ppo_a_r,ppo_r_l  = ppo_agent.calculate_expertise(100)
  a2c_a_r,a2c_r_l   = a2c_agent.calculate_expertise(100)
  all_rewards[0].append(dqn_r_l)
  all_rewards[1].append(ppo_r_l)
  all_rewards[2].append(a2c_r_l)



Using cpu device
Using cpu device
Using cpu device
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.99     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 8871     |
|    time_elapsed     | 0        |
|    total_timesteps  | 102      |
----------------------------------
xxx
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.972    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 7119     |
|    time_elapsed     | 0        |
|    total_timesteps  | 294      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.969    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 6536     |
|    time_elapsed     | 0        |
|    total_timesteps  | 330      |
----------------------------------


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
|    total_timesteps  | 86399    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 4.57e-05 |
|    n_updates        | 9099     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 924      |
|    fps              | 1861     |
|    time_elapsed     | 46       |
|    total_timesteps  | 86470    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.00842  |
|    n_updates        | 9117     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 928      |
|    fps              | 1852     |
|    time_elapsed     | 46       |
|    total_timesteps  | 86887    |
| tr

In [1]:
alls_3=[]
for alls_1 in all_rewards[0]:
    t=0
    list_1=[]
    for i in range(100):
        t+=alls_1[i]
        list_1.append(t/(i+1))
    alls_3.append(list_1)
    rewards=alls_3
mean_regret = np.mean(rewards, axis=0)
std_regret = np.std(rewards, axis=0)

upper_bound = (mean_regret + 1.96 * std_regret / np.sqrt(len(rewards))).flatten()
lower_bound = (mean_regret - 1.96 * std_regret / np.sqrt(len(rewards))).flatten()

fig, ax = plt.subplots()
ax.plot(mean_regret, label="DQN")
ax.fill_between(np.arange(len(mean_regret)).flatten(), lower_bound ,upper_bound , alpha=0.5)

alls_3=[]
for alls_1 in all_rewards[1]:
    t=0
    list_1=[]
    for i in range(100):
        t+=alls_1[i]
        list_1.append(t/(i+1))
    alls_3.append(list_1)
    rewards=alls_3
mean_regret = np.mean(rewards, axis=0)
std_regret = np.std(rewards, axis=0)

upper_bound = (mean_regret + 1.96 * std_regret / np.sqrt(len(rewards))).flatten()
lower_bound = (mean_regret - 1.96 * std_regret / np.sqrt(len(rewards))).flatten()
print(lower_bound)

# fig, ax = plt.subplots()
ax.plot(mean_regret, label="PPO")
ax.fill_between(np.arange(len(mean_regret)).flatten(), lower_bound ,upper_bound , alpha=0.5)


alls_3=[]
for alls_1 in all_rewards[2]:
    t=0
    list_1=[]
    for i in range(100):
        t+=alls_1[i]
        list_1.append(t/(i+1))
    alls_3.append(list_1)
    rewards=alls_3
mean_regret = np.mean(rewards, axis=0)
std_regret = np.std(rewards, axis=0)

upper_bound = (mean_regret + 1.96 * std_regret / np.sqrt(len(rewards))).flatten()
lower_bound = (mean_regret - 1.96 * std_regret / np.sqrt(len(rewards))).flatten()
print(lower_bound)

# fig, ax = plt.subplots()
ax.plot(mean_regret, label="A2C")
ax.fill_between(np.arange(len(mean_regret)).flatten(), lower_bound ,upper_bound , alpha=0.5)

ax.plot([0.38 for i in range(100)],label='Expected hit count for random agent')

plt.legend()
# Show the plot
plt.show()

# plt.legend()
# # Show the plot
# plt.show()


NameError: name 'all_rewards' is not defined

In [32]:
import pickle
file_path = "./all_rewards_for_100000_timesteps.pkl"

# Open the file in binary write mode
with open(file_path, "wb") as f:
    # Serialize and save the object to the file
    pickle.dump(all_rewards, f)
